In [1]:
import pandas as pd
import json
import numpy as np
import os
from google.cloud import storage
from io import BytesIO
from pypdf import PdfReader 
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from langchain_google_vertexai import ChatVertexAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import logging
from dotenv import load_dotenv

In [2]:
load_dotenv()
json_path = os.getenv('json_path') # Variable donde se encuentra la ruta del json con las credenciales de Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_path 
logging.basicConfig(filename='error_log.txt', level=logging.ERROR, format='%(asctime)s %(message)s')
bucket_name = 'tfm_javi'

In [41]:
# Función que se encarga de devolver un PDF dado su nombre y el bucket donde está almacenado
def leer_blob_en_memoria(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    pdf_content = blob.download_as_bytes()
    return pdf_content

# Función que transforma un pdf en una cadena de texto que puede ser procesada
def extraer_texto_de_pdf_bytes(pdf_bytes):
    pdf_file = BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

# Función que combina las dos funciones creadas anteriormente en una sola
def procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name):
    pdf_bytes = leer_blob_en_memoria(bucket_name, blob_name)
    content = extraer_texto_de_pdf_bytes(pdf_bytes)
    return content

# Función que se encarga de devolver una lista con todos los nombres de los archivos en un bucket
def listar_pdfs(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()

    # Filtra los archivos PDF y guarda sus URLs completas en la lista
    listado_pdfs = [f"gs://{bucket_name}/{blob.name}" for blob in blobs if blob.name.endswith('.pdf') and not blob.name.startswith('resoluciones/')]
    return listado_pdfs

# Función que se encarga de dividir la lista de todos los PDFs en 5 partes para facilitar su procesado en tandas
def dividir_lista_pdfs(listado_pdfs):
    total = len(listado_pdfs)
    tamaño_parte = total // 5
    
    parte1 = listado_pdfs[:tamaño_parte]
    parte2 = listado_pdfs[tamaño_parte:2*tamaño_parte]
    parte3 = listado_pdfs[2*tamaño_parte:3*tamaño_parte]
    parte4 = listado_pdfs[3*tamaño_parte:4*tamaño_parte]
    parte5 = listado_pdfs[4*tamaño_parte:]
    
    return parte1, parte2, parte3, parte4, parte5

In [7]:
# Definición los esquemas de respuesta
response_schemas = [
    ResponseSchema(name="numero_expediente", description="Número de Expediente, ejemplo SD2023/0000046"),
    ResponseSchema(name="resolucion", description="Una de las siguientes opciones: negada_con_oposicion, negada_sin_oposicion, aprobada_sin_oposicion, aprobada_con_oposición"),
    ResponseSchema(name="numero_de_resolución", description="Número entero de la resolución, ejemplo 2195"),
    ResponseSchema(name="denominacion", description="Nombre de la empresa que solicita el registro de la marca"),
    ResponseSchema(name="vigencia", description="Fecha en que expira la vigencia del registro, o texto si es negada o vencida"),
    ResponseSchema(name="titular", description="Titular de la marca que intenta registrar"),
    ResponseSchema(name="clase", description="Número o lista de números de la Clasificación Internacional de Niza, por ejemplo [42, 35, 27]"),
    ResponseSchema(name="gaceta", description="Número de la gaceta de Propiedad Industrial donde se publica"),
    ResponseSchema(name="tipo", description="Tipo de registro que se intenta realizar, por ejemplo Mixta, Nominativa, Figurativa"),
    ResponseSchema(name="fecha_solicitud", description="Fecha de presentación de la solicitud"),
    ResponseSchema(name="fecha_resolucion", description="Fecha de resolución"),
    ResponseSchema(name="nombre_opositor", description="Nombre de la empresa que se opone a la publicación"),
    ResponseSchema(name="signo_opositor_opositores", description="Signo o signos de los opositores en conflicto"),
    ResponseSchema(name="argumento_oposición", description="Argumentos en los que se basa para oponerse al registro y artículos en los que se apoya"),
    ResponseSchema(name="explicacion_argumentos_oposicion", description="Breve resumen y explicación de los argumentos de la oposición"),
    ResponseSchema(name="resolucion_organismo", description="Resolución del organismo competente, por ejemplo: 'DENIEGA el registro de la marca PAPELES LA FAVORITA (Mixta)'")
]


In [8]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# Escapar las llaves para evitar la interpretación de variables
format_instructions = format_instructions.replace("{", "{{").replace("}", "}}")

In [9]:
# Definición de los prompts que se le van a pasar a VertexAI para darle instrucciones de qué hacer
system_prompt = """
Eres un Experto abogado Colombiano en analizar resoluciones del SIC (Superintendencia de Industria y Comercio de Colombia),en el ámbito de registro de marcas y lemas.
quiero que extraigas el numero de Expediente, la resolución del conflicto, el numero de la resolución, el nombre de la marca que intenta registrarse, el titular que intenta registrar la marca, el numero de clase que intenta registrar, 
el numero de la gaceta en que ha sido publicada, la fecha de solicitud de registro, nombre de la empresa opositora, el titular de la empresa que se opone si aparece, y los argumentos de derecho en los que se apoya el opositor.
"""
human_prompt = f"""Extrae la información indicada en DATOS a partir del TEXTO de la resolución

TEXT
---
\n\n{{contenido_pdf}}
---

DATOS
{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(human_prompt),
])

In [11]:
# Define tu LLM 
llm = ChatVertexAI(
    model="gemini-1.5-pro-001" 
)

chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)

In [38]:
# Obtener la lista de PDFs
listado_pdfs = listar_pdfs(bucket_name)

# Comprobamos la longitud total de los PDFs en el bucket
len(listado_pdfs)

In [43]:
# Dividimos la lista de PDFs en 5 partes para facilitar su procesado (procesar tantos PDFs lleva tiempo y procesar 12000-13000 PDFs implicaria demasiado tiempo)
parte1, parte2, parte3, parte4, parte5 = dividir_lista_pdfs(listado_pdfs)

In [51]:
# Inicializar una lista para almacenar los resultados y los errores
resultados = []
listado_pdfs_error = []

## Parte 1

A continuación se va a proceder a transformar todos los PDFs de la parte 1 en datos que puedan ser explotados mas adelante. Durante el procesado de estos PDFs a veces se pueden dar errores, por lo que guardamos los nombres de los archivos que han dado error para poder procesarlos de nuevo mas adelante. Una vez se haya concluido de procesar la parte 1, se hará lo mismo con el resto de las partes (2, 3, 4, y 5)

In [ ]:
# Procesamos todos los PDFs de la primera parte
for pdf_uri in parte1:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 1

Todos los PDFs que dieron error al ser procesados en la sección anterior se han fuardado en una variable llamada listado_pdfs_error. Para procesar esta parte procedemos a llamar a la misma función que en el apartado anterior

In [57]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)    

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [60]:
df_resultados = pd.DataFrame(resultados)

In [ ]:
df_resultados

In [62]:
pd.set_option('display.max_columns', None)

In [63]:
df_ai_annotations_full_1 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
df_ai_annotations_full_1

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_1.info()

In [67]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_1.to_csv("datasetia_full_1.csv")

A continuación vamos a realizar los mismo pasos para las partes 2, 3, 4, y 5

## Parte 2

In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte2:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 2

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)    

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_2 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_2.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_2.to_csv("datasetia_full_2.csv")

## Parte 3


In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte3:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 3

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)    

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_3 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_3.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_3.to_csv("datasetia_full_3.csv")

## Parte 4

In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte4:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 4

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)   

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_4 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_4.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_4.to_csv("datasetia_full_4.csv")

## Parte 5

In [ ]:
# Reiniciamos las listas
resultados = []
listado_pdfs_error = []

In [ ]:
# Procesamos todos los PDFs de la segunda parte
for pdf_uri in parte5:
    blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
    contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
    # Verificar si el contenido del PDF no está vacío
    if contenido_pdf.strip():
        try:
            # Invocar la cadena con el contenido del PDF
            res = chain.invoke({"contenido_pdf": contenido_pdf})
            #Agregar el resultado a la lista de resultados
            resultados.append(res)
            #Imprimir el resultado
            print(f"Resultado para {blob_name}:\n{res}\n")
        except Exception as e:
            # Si hay algún error al procesar el archivo (a veces pasa), guardamos su nombre para intentarlo otra vez mas adelante
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            listado_pdfs_error.append(blob_name)
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
    else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

## Errores de analisis de la parte 5

In [ ]:
# Comprobamos la longitud de la lista con los PDFs que han dado error al ser procesados
print(listado_pdfs_error)   

In [ ]:
# Recorremos la lista de nuevo procesando los PDFs una segunda vez
for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

In [ ]:
df_resultados = pd.DataFrame(resultados)
pd.set_option('display.max_columns', None)
df_ai_annotations_full_5 = pd.DataFrame.from_records(df_resultados['text'])

In [ ]:
# Comprobamos cuantas files tienen entradas en blanco y el tipo de variable de cada columna
df_ai_annotations_full_5.info()

In [ ]:
# Guardamos los resultados en un archivo csv
df_ai_annotations_full_5.to_csv("datasetia_full_5.csv")